In [1]:
%run ../../Environment/environment.ipynb

e:\Studienarbeit_GitHub\Studienarbeit\Agents\PPO
Notebook ausgeführt


# 1. Bibliotheken importieren

In [2]:
import numpy as np
import pandas as pd
import optuna
import joblib
import random
from stable_baselines3 import PPO, A2C, DQN
from stable_baselines3.common.callbacks import CheckpointCallback
from sklearn.preprocessing import StandardScaler
from stable_baselines3.common.vec_env import DummyVecEnv
import torch

# 2. Seed setzen

In [3]:
seed = 42
SEED  = seed % (2**32 - 1)
print(f"SEED: {SEED}")

SEED: 42


# 3. Daten einlesen

In [4]:
# -------------------------------
# CSV Datem einlesen
# -------------------------------
train_data = pd.read_csv("../../Transform_data/stand_data/2023-2018_stand_data.csv")
train_data.drop('datetime', axis=1, inplace=True)

test_data = pd.read_csv("../../Transform_data/stand_data/2025-2024_stand_data.csv")
test_data.drop('datetime', axis=1, inplace=True)

if(train_data is not None and test_data is not None):
    print("Daten erfolgreich eingelesen")

Daten erfolgreich eingelesen


# 4. Parallele Umgebungen erstellen für das Training

In [5]:
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common.vec_env import VecNormalize

def make_env():
    return TradingEnv(
        data=train_data,
        initial_cash=10_000,
        window_size=336,
        scaler_path="../../Transform_data/scaler.pkl",
        default_seed=SEED
    )

n_envs = 8  # Mehr parallele Umgebungen (8, 16 oder sogar 32 testen!)
env = SubprocVecEnv([make_env for _ in range(n_envs)])

env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.0)
env.training = True  # Sicherstellen, dass Normalisierung aktiv ist

# 5. Hyperparameter Evaluierung

In [9]:
# Für Hyperparameter-Tuning nutzen wir test_data als Validierungsdatensatz.
valid_data = test_data.copy()

# -------------------------------
# Evaluation Helper Function
# -------------------------------
def evaluate_agent(model, env, n_eval_episodes=5):
    """
    Evaluate the model over a number of episodes.
    Returns the average cumulative reward.
    """
    episode_rewards = []
    for _ in range(n_eval_episodes):
        # Handle reset return for compatibility with gym vs. gymnasium APIs
        reset_result = env.reset()
        if isinstance(reset_result, tuple):
            obs, info = reset_result
        else:
            obs = reset_result
        done = False
        total_reward = 0.0
        while not done:
            action, _states = model.predict(obs, deterministic=True)
            step_result = env.step(action)
            # Check length to handle different API outputs
            if len(step_result) == 5:
                obs, reward, done, truncated, info = step_result
            else:
                obs, reward, done, info = step_result
            total_reward += reward
        episode_rewards.append(total_reward)
    return np.mean(episode_rewards)

In [14]:
# -------------------------------
# Hyperparameter Tuning with Optuna
# -------------------------------
def objective(trial):
    # Sample hyperparameters using updated Optuna functions
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
    gamma = trial.suggest_float("gamma", 0.90, 0.9999)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128, 256])
    n_steps = trial.suggest_categorical("n_steps", [128, 256, 512, 1024])
    ent_coef = trial.suggest_float("ent_coef", 1e-6, 0.01, log=True)
    clip_range = trial.suggest_float("clip_range", 0.1, 0.4)
    gae_lambda = trial.suggest_float("gae_lambda", 0.8, 0.99)

    
    # Create the training environment using train_data
    env_train = DummyVecEnv([lambda: TradingEnv(
        data=train_data,
        initial_cash=10_000,
        window_size=336,
        scaler_path="../../Transform_data/scaler.pkl",
        default_seed=SEED
    )])
    
    # Create the validation environment using valid_data
    env_valid = DummyVecEnv([lambda: TradingEnv(
        data=valid_data,
        initial_cash=10_000,
        window_size=336,
        scaler_path="../../Transform_data/scaler.pkl",
        default_seed=SEED
    )])
    
    # Initialize the PPO model with sampled hyperparameters
    model = PPO(
        "MlpPolicy",
        env_train,
        learning_rate=learning_rate,
        gamma=gamma,
        batch_size=batch_size,
        n_steps=n_steps,
        ent_coef=ent_coef,
        clip_range=clip_range,
        gae_lambda=gae_lambda,
        verbose=0,
        seed=SEED,
    )
    
    # Train the model for a fixed number of timesteps
    model.learn(total_timesteps=10000, log_interval=1)
    
    # Evaluate the trained model on the validation environment
    mean_reward = evaluate_agent(model, env_valid, n_eval_episodes=5)
    return mean_reward

In [15]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)
print("Best hyperparameters:", study.best_trial.params)

[I 2025-04-06 17:11:36,961] A new study created in memory with name: no-name-41726fc8-e193-4554-97fc-8523bae310ca


Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42


[I 2025-04-06 17:13:18,187] Trial 0 finished with value: 0.0 and parameters: {'learning_rate': 0.0008541333189903621, 'gamma': 0.9663429213168511, 'batch_size': 32, 'n_steps': 256, 'ent_coef': 0.00011614601472383898, 'clip_range': 0.31303180109881346, 'gae_lambda': 0.894724532868429}. Best is trial 0 with value: 0.0.


Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42


[I 2025-04-06 17:14:59,561] Trial 1 finished with value: 0.0 and parameters: {'learning_rate': 0.0018022533807442135, 'gamma': 0.9945711045838953, 'batch_size': 32, 'n_steps': 128, 'ent_coef': 0.00028934493037223826, 'clip_range': 0.3585110258482842, 'gae_lambda': 0.9474647455947949}. Best is trial 0 with value: 0.0.


...

Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42


[I 2025-04-06 18:27:51,601] Trial 48 finished with value: -7.498857021331787 and parameters: {'learning_rate': 0.00018221383852095463, 'gamma': 0.9613084670291221, 'batch_size': 128, 'n_steps': 128, 'ent_coef': 0.005355701723210471, 'clip_range': 0.3058243957907378, 'gae_lambda': 0.9446250663941415}. Best is trial 0 with value: 0.0.


Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42
Seed in the environment: 42


[I 2025-04-06 18:29:28,017] Trial 49 finished with value: -7.507968902587891 and parameters: {'learning_rate': 0.001738987496241959, 'gamma': 0.999883694576279, 'batch_size': 32, 'n_steps': 128, 'ent_coef': 5.834324105722164e-05, 'clip_range': 0.27381516451977883, 'gae_lambda': 0.9353431271065653}. Best is trial 0 with value: 0.0.


Seed in the environment: 42
Best hyperparameters: {'learning_rate': 0.0008541333189903621, 'gamma': 0.9663429213168511, 'batch_size': 32, 'n_steps': 256, 'ent_coef': 0.00011614601472383898, 'clip_range': 0.31303180109881346, 'gae_lambda': 0.894724532868429}


Best hyperparameters: {'learning_rate': 0.0008541333189903621, 'gamma': 0.9663429213168511, 'batch_size': 32, 'n_steps': 256, 'ent_coef': 0.00011614601472383898, 'clip_range': 0.31303180109881346, 'gae_lambda': 0.894724532868429}

# 6. Erstellen des Agenten

In [ ]:
from torch import nn  # Für die Netzwerkarchitektur

# Definiere das neuronale Netz
policy_kwargs = dict(
    net_arch=[dict(pi=[128, 128], vf=[128, 128])],  # Zwei Layer mit 128 Neuronen
    activation_fn=nn.ReLU,  # Verwende ReLU als Aktivierungsfunktion
)

# Erstelle den PPO-Agenten ohne Hyperparametern
model_without = PPO("MlpPolicy", env, seed=SEED, verbose=1)


# Erstelle den PPO-Agenten mit Optuna-Hyperparametern
model_optuna = PPO(
    "MlpPolicy",
    env,
    learning_rate=0.0008541333189903621,
    gamma=0.9663429213168511,
    batch_size=32,
    n_steps=256,
    ent_coef=0.00011614601472383898,
    clip_range=0.31303180109881346,
    gae_lambda=0.894724532868429,
    policy_kwargs=policy_kwargs,
    verbose=1,
    seed=SEED,
    device="cuda",
)

# 7. Modell trainieren und speichern

In [ ]:
model.learn(total_timesteps=10000)

model.save("ppo_trading_model")